In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Concatenate , Add, Dot, Activation, Lambda, BatchNormalization, LeakyReLU, Softmax, ReLU
from tensorflow.keras.models import Model

from tensorflow.image import flip_up_down, flip_left_right, rot90
from tensorflow.linalg import normalize

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import math

import matplotlib.pyplot as plt
import sys 
sys.path.append('..')
from layers.sorted_filters_with_shift import SortedConv2DWithShift

import datetime


In [2]:
from tensorflow.python.client import device_lib
#tf.disable_v2_behavior()

#print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

print(tf.__version__ )

Num GPUs Available:  1
2.9.1


In [3]:
input_net = Input((32, 32, 3))

y, map = SortedConv2DWithShift(filters=10 , activation=ReLU(), padding='SAME')(input_net)

'''y = layers.Flatten()(shifted)
y = layers.Dense(200, activation=ReLU())(y)
y = layers.BatchNormalization(axis=-1)(y)
output_net  = layers.Dense(100, activation=Softmax())(y)'''

model = Model(inputs = input_net, outputs = y)
model.summary()

here
doing
False


StagingError: Exception encountered when calling layer "sorted_conv2d_with_shift" (type SortedConv2DWithShift).

in user code:

    File "c:\Users\AN46710\Desktop\ri_feature_cnn\Models\layers\sorted_filters_with_shift.py", line 158, in call  *
        shifted, map = self.shift_to_max(x, x_a)
    File "c:\Users\AN46710\Desktop\ri_feature_cnn\Models\layers\sorted_filters_with_shift.py", line 224, in shift_to_max  *
        shifted = self.roll(x, -1*map)
    File "c:\Users\AN46710\Desktop\ri_feature_cnn\Models\layers\sorted_filters_with_shift.py", line 182, in roll_fn  *
        out = tf.zeros([x.shape[0], x.shape[1], x.shape[2], x.shape[3]])

    IndexError: tuple index out of range


Call arguments received by layer "sorted_conv2d_with_shift" (type SortedConv2DWithShift):
  • inputs=tf.Tensor(shape=(None, 32, 32, 3), dtype=float32)
  • training=None

In [ ]:
num_classes = 100
input_shape = (32, 32, 3)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()



# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
#x_train = np.expand_dims(x_train, -1)
#x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

plt.imshow(x_train[0,:,:,:])



In [ ]:
model = Model(inputs = input_net, outputs = y)

initial_learning_rate =  0.01

lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate,
    alpha = 1e-6,
    m_mul=0.999,
    first_decay_steps = 20)

optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)

log_dir = "logs_shift_experiments/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
)


checkpoint_filepath = "/tmp/checkpoint"
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
)

batch_size = 2
epochs = 100
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), callbacks=[tensorboard_callback])

In [ ]:
%tensorboard --logdir logs_shift_experiments/fit

In [ ]:
@tf.function
def roll(arg):
    x, map = arg
    shifted = tf.TensorArray(tf.float32, size=map.shape[-1], dynamic_size=False)
    print('here', map.shape)
    print('here', x.shape)

    for i in range(map.shape[-1]):
        tmp = tf.roll(x[i], shift=map[i], axis=-1)
        shifted = shifted.write(i, tmp)  # shape is (n_atoms, n_timesteps, n_atoms, n_atoms)
    out = shifted.stack()
    print(out.shape)
    return out


get_layer_output = K.function([model.input], model.layers[1].output)

n = 109

X = x_test[n]
print(X.shape)

out = np.array(get_layer_output(np.expand_dims(X, axis=0))[0])
#shifted = np.array(get_layer_output(np.expand_dims(X, axis=0))[1])
map = np.array(get_layer_output(np.expand_dims(X, axis=0))[1])
plt.figure(1)  
im = plt.imshow(out[:,0,4])   
plt.colorbar(im, ticks=range(8), orientation="horizontal")

max_channels = map
max_channels  = tf.cast(max_channels, tf.float32)
averages = tf.nn.avg_pool2d((tf.expand_dims(max_channels, axis=-1)), ksize=[4, 4] , strides=[4, 4], padding='SAME')
map = tf.repeat(averages, repeats = 4, axis=1)
map = tf.repeat(map, repeats = 4, axis=2)
map  = tf.cast(map, tf.int32)

out = tf.reshape(out, [-1, out.shape[1]*out.shape[2], out.shape[3] ])
map = tf.reshape(map, [-1, map.shape[1]*map.shape[2]])
print(out.shape)
print(map.shape)
print(tf.range(out.shape[1])[0])
shifted = tf.vectorized_map(roll, (out, -1*map))
print(shifted.shape)

shifted = tf.reshape(shifted, shape=([1, 32, 32, 10]))
print(out.shape)






plt.figure(2)  
im = plt.imshow(shifted[:,0, 4]) 
plt.colorbar(im, ticks=range(8), orientation="horizontal")

print(map[0, 4])




In [ ]:
0

In [ ]:
model.layers[1].gain

In [ ]:
model.layers[5].scale_a